# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [2]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_termos_produto(lista_termos_nome_produto, nome):   
    tem_todos_termos_produto = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False 
    return tem_todos_termos_produto

def busca_google(nav, produto, termos_banidos, preco_minimo, preco_maximo):

    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    resultados = []

    nav.get('https://www.google.com.br')
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    
    time.sleep(6)
    
    elementos = nav.find_elements('class name', 'GKS7s')

    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break

    lista_resultados = nav.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()

        # analisar se nao tem termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # tem todos termos do nome do produto

        tem_todos_termos_produto = verificar_tem_termos_produto(lista_termos_nome_produto, nome)
        

        # selecionar so elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produto = True

        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            preco = float(preco)

            preco_minimo = float(preco_minimo)
            preco_maximo = float(preco_maximo)

            # se o preco ta entre o minimo e o maximo

            if preco_minimo <= preco <= preco_maximo:

                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                resultados.append((preco, nome, link))

    return resultados



def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    resultados = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    nav.get('https://www.buscape.com.br')
    nav.find_element(By.XPATH,
                     '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    time.sleep(4)

    lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        link = resultado.get_attribute('href')

        # analisar se nao tem termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # tem todos termos do nome do produto
        tem_todos_termos_produto = verificar_tem_termos_produto(lista_termos_nome_produto, nome)

        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:  
                resultados.append((preco, nome, link))   
                
    return resultados


In [3]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:

    # buscar o produto
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']

    google = busca_google(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if google:
        tabela_google = pd.DataFrame(google, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google])
    else: 
        tabela_google = None
        
    buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if buscape:
        tabela_buscape = pd.DataFrame(buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else: 
        tabela_buscape = None
    
    
display(tabela_ofertas)

,produto,preco,link
0,3099.00,usado: iphone 12 64gb preto bom - trocafone,https://www.google.com.br/url?url=https://www....
1,3349.00,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",https://www.google.com.br/url?url=https://www....
2,3279.00,usado: iphone 12 64gb azul bom - trocafone,https://www.google.com.br/url?url=https://www....
3,3404.00,iphone 12 64gb usado varias cores + carregador,https://www.google.com.br/url?url=https://www....
4,3349.00,"iphone 12 64gb branco de vitrine tela 6,1&quot...",https://www.google.com.br/url?url=https://www....
5,3412.99,apple iphone 12 4gb/64gb 6.1 azul,https://www.google.com.br/url?url=https://www....
6,3015.00,iphone 12 64gb | celular apple | usado,https://www.google.com.br/url?url=https://www....
7,3465.00,iphone 12 64 gb branco | celular iphone | usado,https://www.google.com.br/url?url=https://www....
8,3495.05,apple iphone 12 64gb 5g - 12mp ios - tela supe...,https://www.google.com.br/url?url=https://www....
9,3379.90,celular apple iphone 12 black 64gb vitrine/sem...,https://www.google.com.br/url?url=https://www....


In [4]:
tabela_ofertas.to_excel('Ofertass.xlsx')

In [ ]:
import win32com.client as win32

if len(tabela_ofertas) > 0:
    
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'tarikhadi7@gmail.com'
    mail.Subject = 'Produtos encontrados na faixa de preço desejada.'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos na oferta de preçø desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.</p>
    """
    
    mail.Send()